In [1]:
import collections
import itertools
import operator

START = ord('A')
END = ord('Z')
ALPHABET = list(map(chr, range(START, END+1)))
LENGTH = len(ALPHABET)
DICTIONARY = ['HELLO', 'THE', 'AND', 'FOR', 'ARE', 'BUT', 'NOT', 'YOU', 'ALL', 'ANY', 'HER', 'HIM']
FREQUENCIES = 'ETAOINSHRDLCUMWFGYPBVKJXQZ'

class Caesar:
    @staticmethod
    def table(i, reverse = False):
        """
        Given offset, rotate the alphabet by offset characters, circularly.
        """
        t = collections.deque(ALPHABET)
        d = 1 if not reverse else -1
        t.rotate(d * i)
        return list(t)

    @staticmethod
    def offset(first, second = START):
        """
        Calculate the difference/offset between the first and second chars (if not ints) or offsets (if ints).
        """
        if not type(first) is int:
            first = ord(first)
        if not type(second) is int:
            second = ord(second)
        return (first - second) % LENGTH

    @staticmethod
    def encipher(cleartext, i, reverse = False):
        """
        Given cleartext msg, encode using caesar cipher of provided offset.
        """
        t = Caesar.table(i, reverse=reverse)
        r = []
        for c in cleartext.upper():
            if c in ALPHABET:
                i = Caesar.offset(c)
                r.append(t[i])
        return "".join(r)

    @staticmethod
    def decipher(ciphertext, i):
        """
        Given enciphered cipher, decode using caesar cipher of provided offset.
        """
        return Caesar.encipher(ciphertext, i, reverse=True)
    

class CaesarSolver:
    
    @staticmethod
    def freq(text):
        """
        Count the number of occurences of each character in a given text and return an ordered dictionary.
        """
        d = collections.defaultdict(int)
        for c in text:
            d[c] += 1
        x = sorted(d.items(), key=operator.itemgetter(1))
        for f in x:
            yield f

    @staticmethod
    def freqcrack(ciphertext):
        """
        Given a ciphertext, perform frequency analysis and guess the offset.
        """
        cipherletter, cipherfreq = next(CaesarSolver.freq(ciphertext))
        for clearletter in FREQUENCIES:
            i = Caesar.offset(cipherletter, clearletter)
            cleartext = Caesar.decipher(ciphertext, i)
            yield {'offset': i, 'letter': clearletter, 'cleartext': cleartext, 'ciphertext': ciphertext, 'width': len(str(LENGTH))}

    @staticmethod
    def dictcrack(ciphertext):
        """
        Attempt to crack the caesar ciphered text by looking for words in dictionary.
        """
        for i in range(0, LENGTH):
            for word in DICTIONARY:
                cleartext = Caesar.decipher(ciphertext, i)
                if word in cleartext:
                    yield {'offset': i, 'cleartext': cleartext, 'word': word, 'ciphertext': ciphertext}
                    break

    @staticmethod
    def crack(ciphertext):
        """
        Composite crack function based on frequency analysis and dictionary words.
        """
        for guess in CaesarSolver.freqcrack(ciphertext):
            for word in DICTIONARY:
                if word in guess['cleartext']:
                    yield dict(guess, word=word)
                    break
                    

class Vigenere(Caesar):
    
    @staticmethod
    def encipher(cleartext, password, reverse = False):
        pg = itertools.cycle(password.upper())
        r = []
        for c in cleartext.upper():
            if c in ALPHABET:
                i = Caesar.offset(next(pg))
                r.append(Caesar.encipher(c, i, reverse=reverse))
        return "".join(r)
    
    @staticmethod
    def decipher(ciphertext, password):
        return Vigenere.encipher(ciphertext, password, reverse=True)

    
def gcd(a, b):
    """
    Calculate the gcd of a and b using Euclid's method, iteratively.
    """
    c = 0
    while b != 0:
        a, b = b, a % b
        c += 1
    a = abs(a)
    if count:
        return a, c
    return a


def lcm(a, b):
    """
    Returns the least common multiple of two numbers using gcd.
    """
    return a * b / gcd(a, b)
    
    
class VigenereSolver(CaesarSolver):
    
    @staticmethod
    def dups(text, threshold):
        l = len(text)
        d = collections.defaultdict(int)
        for i in range(l // 2, 0, -1):
            for j in range(l):
                k = j + i
                if k <= l:
                    x = text[j:k]
                    d[x] += 1
        r = dict((k, v) for k, v in d.items() if len(k) >= threshold and v > 1)
        return sorted(r.items(), key=operator.itemgetter(1), reverse=True)

In [2]:
def caesar_guess(ciphertext):
    for g in CaesarSolver.crack(ciphertext):
        print("{offset:{width}}: {cleartext} ({letter}/{word})".format(**g))

def clean(message):
    return "".join([x for x in message.upper() if x.isalpha()])

In [3]:
i = 12
message = 'This is a story about a girl who tried to drown the whole world.'
ciphertext = Caesar.encipher(message, i)
print("Cipher({}):\t{}".format(i, ciphertext))
cleartext = Caesar.decipher(ciphertext, i)
print("Clear({}):\t{}".format(i, cleartext))
assert clean(message) == cleartext

Cipher(12):	HVWGWGOGHCFMOPCIHOUWFZKVCHFWSRHCRFCKBHVSKVCZSKCFZR
Clear(12):	THISISASTORYABOUTAGIRLWHOTRIEDTODROWNTHEWHOLEWORLD


In [4]:
caesar_guess(ciphertext)

12: THISISASTORYABOUTAGIRLWHOTRIEDTODROWNTHEWHOLEWORLD (P/THE)
 2: JXYIYIQIJEHOQREKJQWYHBMXEJHYUTJETHEMDJXUMXEBUMEHBT (Z/THE)


In [5]:
message = 'This is a story about a girl who tried to drown the whole world.'
password = 'shoe'
ciphertext = Vigenere.encipher(message, password)
print("Cipher({}):\t{}".format(password, ciphertext))
cleartext = Vigenere.decipher(ciphertext, password)
print("Clear({}):\t{}".format(password, cleartext))
assert clean(message) == cleartext

Cipher(shoe):	BAUOQLMOBHDUIUAQBTSEZEIDWMDEMWFKLKASVMTAEAAHMPANTW
Clear(shoe):	THISISASTORYABOUTAGIRLWHOTRIEDTODROWNTHEWHOLEWORLD
